In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1012"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 6    #number of convolution to have in the network
pooling_stride = [32, 1, 1, 1, 1, 1]
weights_shape = [[32, n_variables, 32],
                 [1, 32, 32],
                 [1, 32, 32],
                 [1, 32, 32],
                 [1, 32, 32],
                 [1, 32, 32]]
bias_shape = [32, 32, 32, 32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.211469, fmeasure_max= 0.399189, precision= 0.252026, recall= 0.959416
Train Random guess:  , auc= 0.235936fmeasure_max= 0.390120, precision= 0.242329, recall= 1.000000
Validation Iter 0, auc= 0.261218, fmeasure_max= 0.373489, precision= 0.241675, recall= 0.821612
Validation Random guess:  auc= 0.237255, fmeasure_max= 0.371101, precision= 0.227823, recall= 1.000000
Minibatch Loss= 1.071293
Tarin Iter 100, auc= 0.198372, fmeasure_max= 0.383252, precision= 0.238162, recall= 0.980695
Train Random guess:  , auc= 0.207145fmeasure_max= 0.336582, precision= 0.202344, recall= 1.000000
Validation Iter 100, auc= 0.270695, fmeasure_max= 0.414986, precision= 0.262654, recall= 0.987993
Validation Random guess:  auc= 0.232067, fmeasure_max= 0.372707, precision= 0.229398, recall= 0.993139
Minibatch Loss= 0.982044
Tarin Iter 200, auc= 0.436445, fmeasure_max= 0.448562, precision= 0.309140, recall= 0.817052
Train Random guess:  , auc= 0.219097fmeasure_max= 0.360551, precision= 0.2199

In [14]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

ValueError: Parent directory of ../model/CNN1012/CNN1012 doesn't exist, can't save.

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [15]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.303571 F1_max = 0.418561 prec = 0.285495 rec = 0.783953
Test Random guess: auc= 0.232137 fmeasure_max= 0.375635 , precision= 0.231250 , recall= 1.000000


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})